In [4]:




import os
import re
import sys
from collections import Counter
import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from scipy.stats import spearmanr
import matplotlib.lines as mlines
# Local application imports
sys.path.insert(0, '../Utility files')
import utils2
from utils2 import *

from model_comparison import CrossValidation

import statsmodels.api as sm
from statsmodels.robust.robust_linear_model import RLM
import importlib
importlib.reload(utils2)

<module 'utils2' from '/home/livtollanes/10.jan-thesis/Code/Validation/../Utility files/utils2.py'>

In [5]:
# Loading CA job coord files


# Load the coordinate files for models 1 to 9
dfs = utils2.load_CA_model_files(9)

for i, df in enumerate(dfs):
    print(f"Shape of dataframe {i+1}: {df.shape}")

Used file path: /home/livtollanes/NewData/job_title_coordinates/m1_jobs_rowcoords.csv
Used file path: /home/livtollanes/NewData/job_title_coordinates/m2_jobs_rowcoords.csv
Used file path: /home/livtollanes/NewData/job_title_coordinates/m3_jobs_rowcoords.csv
Used file path: /home/livtollanes/NewData/job_title_coordinates/m4_jobs_rowcoords.csv
Used file path: /home/livtollanes/NewData/job_title_coordinates/m5_jobs_rowcoords.csv
Used file path: /home/livtollanes/NewData/job_title_coordinates/m6_jobs_rowcoords.csv
Used file path: /home/livtollanes/NewData/job_title_coordinates/m7_jobs_rowcoords.csv
Used file path: /home/livtollanes/NewData/job_title_coordinates/m8_jobs_rowcoords.csv
Used file path: /home/livtollanes/NewData/job_title_coordinates/m9_jobs_rowcoords.csv
Shape of dataframe 1: (11678, 14)
Shape of dataframe 2: (11677, 14)
Shape of dataframe 3: (10958, 14)
Shape of dataframe 4: (10681, 14)
Shape of dataframe 5: (10621, 14)
Shape of dataframe 6: (10626, 14)
Shape of dataframe 7: 

In [19]:
from sklearn.linear_model import HuberRegressor



outcome = 'Salaire_moyen_en_EQTP'
predictors = ['0', '1', '2']


# Define your predictors and outcome
X = dfs[0][predictors].values
y = dfs[0][outcome].values

# Create a HuberRegressor object
huber = HuberRegressor(epsilon=1.35, max_iter=100, alpha=0.0001, warm_start=False, fit_intercept=True, tol=1e-05)

# Fit the model
huber.fit(X, y)

# Print the coefficients
print("Coefficients:")
print(huber.coef_)

Coefficients:
[-4.96630466e-05  6.67128015e-05  1.58902088e-05]


In [10]:
import statsmodels.api as sm

# Assume dfs[0] is your DataFrame
X = dfs[0][predictors]
y = dfs[0][outcome]

X = sm.add_constant(X)  # Add a constant column to include an intercept in the model
model = sm.OLS(y, X)
results = model.fit()

# Get robust covariance results with HC3
robust_results = results.get_robustcov_results(cov_type='HC3')

print(robust_results.summary())

                              OLS Regression Results                             
Dep. Variable:     Salaire_moyen_en_EQTP   R-squared:                       0.006
Model:                               OLS   Adj. R-squared:                  0.005
Method:                    Least Squares   F-statistic:                     21.27
Date:                   Thu, 30 May 2024   Prob (F-statistic):           9.81e-14
Time:                           09:44:29   Log-Likelihood:                -99560.
No. Observations:                  11678   AIC:                         1.991e+05
Df Residuals:                      11674   BIC:                         1.992e+05
Df Model:                              3                                         
Covariance Type:                     HC3                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3697.3664 

In [22]:
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Define your predictors and outcome
X = dfs[0][predictors]
y = dfs[0][outcome]

X = sm.add_constant(X)  # Add a constant column to include an intercept in the model

# Initial OLS fit
initial_model = sm.OLS(y, X)
initial_results = initial_model.fit()

# Calculate weights from residuals of initial fit
weights = 1.0 / np.abs(initial_results.resid)

# WLS fit with weights
model = sm.WLS(y, X, weights=weights)
results = model.fit()

# Get robust covariance results with HC3
robust_results = results.get_robustcov_results(cov_type='HC3')

# Print the summary
print("Summary for the first dataframe:")
print(results.summary())

# Calculate the mean squared error and mean absolute error 
y_pred = results.predict(X)
mse = mean_squared_error(y, y_pred) 
mae = mean_absolute_error(y, y_pred) 

print("WLS MSE:", mse) 
print("WLS MAE:", mae) 

Summary for the first dataframe:
                              WLS Regression Results                             
Dep. Variable:     Salaire_moyen_en_EQTP   R-squared:                       0.110
Model:                               WLS   Adj. R-squared:                  0.110
Method:                    Least Squares   F-statistic:                     480.9
Date:                   Thu, 30 May 2024   Prob (F-statistic):          1.25e-294
Time:                           10:08:23   Log-Likelihood:                -88400.
No. Observations:                  11678   AIC:                         1.768e+05
Df Residuals:                      11674   BIC:                         1.768e+05
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [23]:
from sklearn.model_selection import KFold
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Define your predictors and outcome
X = dfs[0][predictors].values
y = dfs[0][outcome].values

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

mse_scores = []
mae_scores = []

# K-fold Cross Validation model evaluation
for train, test in kfold.split(X, y):

    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

    X_train = sm.add_constant(X_train)  # Add a constant column to include an intercept in the model

    # Initial OLS fit
    initial_model = sm.OLS(y_train, X_train)
    initial_results = initial_model.fit()

    # Calculate weights from residuals of initial fit
    weights = 1.0 / np.abs(initial_results.resid)

    # WLS fit with weights
    model = sm.WLS(y_train, X_train, weights=weights)
    results = model.fit()

    # Get robust covariance results with HC3
    robust_results = results.get_robustcov_results(cov_type='HC3')

    # Make predictions on the test data
    X_test = sm.add_constant(X_test)
    y_pred = results.predict(X_test)

    # Calculate the mean squared error and mean absolute error 
    mse = mean_squared_error(y_test, y_pred) 
    mae = mean_absolute_error(y_test, y_pred) 

    mse_scores.append(mse)
    mae_scores.append(mae)

print("Average MSE after CV:", np.mean(mse_scores)) 
print("Average MAE after CV:", np.mean(mae_scores)) 

Average MSE after CV: 1496168.7597543069
Average MAE after CV: 644.7494601645492


In [30]:
from sklearn.model_selection import KFold
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define your predictors and outcome
X = dfs[3][predictors].values
y = dfs[3][outcome].values

# Define the K-fold Cross Validator
kf = KFold(n_splits=5)

mse_scores = []
mae_scores = []
r2_scores = []

# Add a constant column to include an intercept in the model
X = sm.add_constant(X)

# Initial OLS fit
initial_model = sm.OLS(y, X)
initial_results = initial_model.fit()

# Calculate weights from residuals of initial fit
weights = 1.0 / np.abs(initial_results.resid)

# WLS fit with weights
model = sm.WLS(y, X, weights=weights)
results = model.fit()

# Print the summary
print("Summary for the full data fit:")
print(results.summary())

# Calculate the R-squared for the full data fit
print("R-squared for the full data fit:", results.rsquared)

# Calculate the RMSE and MAE for the full data fit
y_pred = results.predict(X)
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)
print("RMSE for the full data fit:", np.sqrt(mse))
print("MAE for the full data fit:", mae)

# K-fold Cross Validation model evaluation
for train, test in kf.split(X):

    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

    # Initial OLS fit
    initial_model = sm.OLS(y_train, X_train)
    initial_results = initial_model.fit()

    # Calculate weights from residuals of initial fit
    weights = 1.0 / np.abs(initial_results.resid)

    # WLS fit with weights
    model = sm.WLS(y_train, X_train, weights=weights)
    results = model.fit()

    # Make predictions on the test data
    y_pred = results.predict(X_test)

    # Calculate the mean squared error, mean absolute error and R-squared
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mse_scores.append(mse)
    mae_scores.append(mae)
    r2_scores.append(r2)

print("Average RMSE after CV:", np.sqrt(np.mean(mse_scores)))
print("Average MAE after CV:", np.mean(mae_scores))
print("Average R-squared after CV:", np.mean(r2_scores))

Summary for the full data fit:
                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.059
Model:                            WLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     222.3
Date:                Thu, 30 May 2024   Prob (F-statistic):          6.46e-140
Time:                        10:26:45   Log-Likelihood:                -80552.
No. Observations:               10681   AIC:                         1.611e+05
Df Residuals:                   10677   BIC:                         1.611e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3588.5386